Работа с языковыми моделями﻿

In [17]:
# | output: false
# | echo: false

from langchain_gigachat.chat_models import GigaChat
from GIGACHAT_CREDENTIALS import Scope

model = GigaChat(
    credentials=Scope,
    scope="GIGACHAT_API_PERS",
    model="GigaChat",
    verify_ssl_certs=False,
)

In [18]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Привет! Меня зовут Вася")])

AIMessage(content='Привет, Вася! Чем могу быть полезен?', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 20, 'completion_tokens': 13, 'total_tokens': 33}, 'model_name': 'GigaChat:1.0.26.20', 'x_headers': {'x-request-id': '8dc3fd95-5dfa-43d0-acfd-04657415ec24', 'x-session-id': '9a08f706-9e78-49ce-8bb9-abaf51d8146d', 'x-client-id': None}, 'finish_reason': 'stop'}, id='8dc3fd95-5dfa-43d0-acfd-04657415ec24')

In [19]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Привет! Меня зовут Вася"),
        AIMessage(
            content="Здравствуйте, Вася! Я генеративная языковая модель от Сбера. Готова ответить на ваши вопросы."
        ),
        HumanMessage(content="Как меня зовут?"),
    ]
)

AIMessage(content='Вас зовут Вася.', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 65, 'completion_tokens': 8, 'total_tokens': 73}, 'model_name': 'GigaChat:1.0.26.20', 'x_headers': {'x-request-id': 'b0e08607-d171-4f0e-858c-edc85a1c15cc', 'x-session-id': 'c9e671ee-949c-43ef-be2b-ecd16bd50ecb', 'x-client-id': None}, 'finish_reason': 'stop'}, id='b0e08607-d171-4f0e-858c-edc85a1c15cc')

In [24]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Инизиализируйте граф
workflow = StateGraph(state_schema=MessagesState)

# Определите функцию, которая вызывает модель
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

# Задайте вершину графа
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Добавьте память
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc123"}}

In [ ]:
query = "Привет! Меня зовут Вася."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # вывод содержит все сообщения в состоянии

In [ ]:
query = "Как меня зовут?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

In [21]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Ты личный помощник. Старайся как можно лучше помочь пользователю.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [22]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    chain = prompt | model
    response = chain.invoke(state)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [23]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Привет! Меня зовут Кира."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Привет, Кира! Как я могу помочь тебе сегодня?


In [25]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Ты — личный ассистент. Старайся как можно лучше помочь пользователю. Отвечай на все вопросы пользователя на следующем языке: {language}. Не называй своего имени.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [26]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    chain = prompt | model
    response = chain.invoke(state)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [29]:
config = {"configurable": {"thread_id": "abc789"}}
query = "Привет! Я Вася. Расскажи историю из 1000 слов"
language = "English"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

Привет, Вася!| Вот тебе история о маленьком мальчике по имени Алеша, который жил в небольшом городке на берегу моря.| Его родители были рыбаками, и с самого детства Алеша помогал им на их лодках.| Каждый день он наблюдал за морем, его волнами и штормами, и мечтал узнать, что же скрывается за горизонтом.|

Однажды, когда Алеше было всего десять лет, он нашел старую карту сокровищ, спрятанную в сундуке у деда.| На карте была изображена таинственная пещера на одном из островов неподалеку.| Алеша решил во что бы то ни стало найти эту пещеру и отправился в путь на своей маленькой лодке.|

Много дней он плыл по морю, преодолевая бурные волны и сильные ветры.| Однажды ночью, после долгого плавания, он увидел свет маяка на острове.| Это был знак, что он почти достиг цели.| Утром Алеша высадился на берег и пошел искать пещеру.| Наконец, он нашел ее - огромную, темную и загадочную.| Внутри пещеры было множество древних артефактов и драгоценностей.| Но самое главное - там находилась старинная кни